In [1]:
import sys
import os
import pickle
import glob

import pandas as pd
import numpy as np

In [2]:
sys.path.append("../exp_adaptive_sv/")

## Kaldi feat 

In [3]:
test_trial = pd.read_csv("../../speaker_extraction/verification/keys/mixture_set/trials", delimiter=' ')

enr_ids = test_trial.Enrollment.unique().tolist()
test_ids = test_trial.Test.unique().tolist()

In [4]:
ids = np.concatenate([enr_ids, test_ids]) 
wsj0_df = pd.DataFrame(ids, columns=['id'])
wsj0_df['file'] = wsj0_df.id.apply(lambda x: os.path.join("/dataset/SV_sets/wsj0/fbank64/", x))
# wsj0_df.to_pickle("/dataset/SV_sets/wsj0/wsj0_df.pkl")

In [55]:
enr_wavs = list(map(lambda x: os.path.join("/dataset/SV_sets/wsj0/wav/", x[:3], x+'.wv1'), enr_ids))
test_wavs = list(map(lambda x: os.path.join("/dataset/SV_sets/wsj0/max_mixed/tt/mix/", x+".wav"), test_ids))
wavs = np.concatenate([enr_wavs, test_wavs])

In [56]:
with open("/dataset/SV_sets/wsj0/wav.scp", "w") as f:
    for id, wav in zip(ids, wavs):
        f.write(id+' '+wav+'\n')

In [38]:
wsj0_df['spk'] = wsj0_df.id.apply(lambda x: x[:3])
wsj0_df = wsj0_df.set_index('id')
wsj0_df = wsj0_df.sort_index()

In [39]:
spks = wsj0_df.spk.unique().tolist()
spk2utt_dict = dict()
for spk in spks:
    spk2utt_dict[spk] = ' '.join(wsj0_df[wsj0_df.spk == spk].index.tolist())

with open("/dataset/SV_sets/wsj0//spk2utt", "w") as f:
    for spk, utt in spk2utt_dict.items():
        f.write(spk+" "+utt+"\n")

with open("/dataset/SV_sets/wsj0//utt2spk", "w") as f:
    for idx, row in wsj0_df.iterrows():
        f.write(idx+" "+row.spk+"\n")

## test_trial

In [8]:
sv_key = pickle.load(open("../sv_system/wsj0_embeds/sv_keys.pkl", "rb"))
sv_embeds = np.load("../sv_system/wsj0_embeds/sv_embeds.npy")

In [16]:
id2idx = {row.id:k for k, row in wsj0_df.iterrows()}
enr_idx = test_trial.Enrollment.apply(lambda x: id2idx[x])
test_idx = test_trial.Test.apply(lambda x: id2idx[x])

In [9]:
enr_embeds = sv_embeds[enr_idx]
enr_embeds = enr_embeds / np.linalg.norm(enr_embeds, axis=1, keepdims=True)

In [10]:
test_embeds = sv_embeds[test_idx]
test_embeds = test_embeds / np.linalg.norm(test_embeds, axis=1, keepdims=True)

In [11]:
cosine_score = (enr_embeds * test_embeds).sum(1)

In [12]:
labels = test_trial.Key.tolist()
labels = list(map(lambda x: 1 if x=='target' else 0, labels))

In [40]:
from utils import compute_eer
compute_eer(cosine_score, labels)

eer: 46.83%, fpr: 46.82%, fnr: 46.83%


(0.4683333333333334, 0.46820833333333334, 0.4683333333333334, 0.8536409)

In [53]:
test_trial.Enrollment[:20]

0     445o030e
1     22ho0110
2     050a050v
3     051a050o
4     052a050h
5     053c0115
6     420o030a
7     421a010a
8     422a0105
9     423c020x
10    440c020x
11    441o030w
12    442c020a
13    443c0202
14    444c020l
15    446c020d
16    447c020x
17    420c020u
18    22gc0101
19    22ho010u
Name: Enrollment, dtype: object

In [54]:
(enr_embeds[0] * enr_embeds[3]).sum()

0.8453253